# Install packages

In [ ]:
%pip install langchain_openai

# Aggregating credit risk ratings and justifications by company


In [2]:
import json

with open('../data/task2_json/articles_with_rating.json', 'r') as file:
    data = json.load(file)

In [3]:
company_with_rating = {}
company_with_justification = {}

for article in data:
  companies = article['companies']
  for i, company_name in enumerate(companies):
    if company_name not in company_with_rating:
      company_with_rating[company_name] = []
    if company_name not in company_with_justification:
      company_with_justification[company_name] = []
      
    # Append credit risk rating and justification for the current company
    company_with_rating[company_name].append(article['credit_risk_rating'][i])
    company_with_justification[company_name].append(article['justification'][i])
    

# Get mean rating for each company

In [4]:
company_with_mean_rating = {}

for key, value in company_with_rating.items():
    if value: 
        company_with_mean_rating[key] = sum(value) / len(value)
    else:
        company_with_mean_rating[key] = None

In [5]:
company_with_mean_rating

{'NBC': 3.1538461538461537,
 'Forbes': 2.0,
 'RealClearPolitics': 3.0,
 'Tesla': 2.75,
 'Ford': 7.0,
 'GM': 6.0,
 'Stellantis': 6.571428571428571,
 'Truth Social': 3.75,
 'Turning Point USA': 4.5,
 'Wood Mackenzie': 3.0,
 'Bachan': 2.0,
 'Prelude Growth Partners': 1.8,
 'Whole Foods': 1.8,
 'Amazon': 1.5625,
 'Walmart': 1.0,
 'Sonoma Brands Capital': 1.8,
 'Facebook': 1.0,
 'McCormick': 1.8,
 'Cholula': 2.0,
 'Heinz': 1.6,
 'Tabasco': 1.6,
 'General Motors': 3.142857142857143,
 'Frigidaire': 5.5,
 'National Cash Register': 4.25,
 'Delco Electronics': 5.75,
 'WWE': 2.0,
 'GoFundMe': 2.0,
 'Citadel': 1.8333333333333333,
 'Heritage Foundation': 3.0,
 'Daily Wire': 4.0,
 'Griffin Catalyst': 2.3333333333333335,
 'Citadel Securities': 1.8333333333333333,
 'Bethel Park Skilled Nursing and Rehabilitation Center': 2.6666666666666665,
 'HBO Max': 2.0,
 'Republican National Committee': 4.0,
 'Democratic Party': 4.0,
 'UPS': 2.3333333333333335,
 'Teamsters National Black Caucus': 5.0,
 'AFL-CIO': 

# Get summarised justification for each company

In [ ]:
from langchain_openai import AzureChatOpenAI

with open("../secrets/openai_secrets.json", "r") as file:
  openai_secrets = json.load(file)


llm = AzureChatOpenAI(
    azure_endpoint=openai_secrets["openai_api_base"],
    openai_api_version="2023-03-15-preview",
    deployment_name="gpt-4-32k",
    openai_api_key=openai_secrets["openai_api_key"],
    openai_api_type="azure",
    temperature=0,
)

In [ ]:
from langchain.prompts.prompt import PromptTemplate

template = """
You are an expert financial analyst specializing in credit risk assessment. Summarise the following justification for the company's credit risk rating.

justification:
{justification}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["justification"],
)

chain = prompt | llm

In [ ]:
company_with_summarised_justification = {}

for company, justification in company_with_justification.items():
  if len(justification) > 1:
    result = chain.invoke({"justification": justification})
    company_with_summarised_justification[company] = result.content
  else:
    result = justification[0]
    company_with_summarised_justification[company] = result
    

In [ ]:
company_with_summarised_justification

{'NBC': 'NBC is a well-established media company with a strong reputation and stable financial position, as indicated by its access to high-profile individuals and events, such as an interview with President Biden. This suggests a low credit risk. However, like other traditional broadcasters such as ABC and Fox, NBC is facing challenges due to the rise of streaming services, decline in cable subscriber counts, and changing TV industry. This could potentially impact its future profitability, especially considering the decline in broadcast TV ratings among younger viewers. Despite these challenges, there are no mentioned financial difficulties or controversies that could harm its creditworthiness.',
 'Forbes': "The company's credit risk rating is justified by its strong reputation and stable financial position. Forbes is a well-known and reputable media and publishing company that continues to produce relevant news and articles, indicating a robust operational capacity. The articles do n

In [ ]:
with open('../data/task2_json/company_with_mean_rating_summary_finding.json', 'w', encoding='utf-8') as f:
  json.dump(company_with_mean_rating, f, indent=4)

In [ ]:
with open('../data/task2_json/company_with_summarised_justification_summary_finding.json', 'w', encoding='utf-8') as f:
  json.dump(company_with_summarised_justification, f, indent=4)